In [3]:
# Dependencies and Setup
import hvplot.pandas as hvplot
import pandas as pd
import requests
import matplotlib.pyplot as plt
import numpy as np
import time
from scipy.stats import linregress

# Import API key
from alan_api_keys import geoapify_key

# Import citipy to determine the cities based on latitude and longitude
from citipy import citipy



In [60]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../data/census_data.csv")

#Create data frame from Cenus data provided by Bryan
df = pd.DataFrame(city_data_df)

#selected columns to pull from the main df
columns_to_pull = ['NAME', 'place']

#pulled selected columns
selected_df = df[columns_to_pull]

#Split the city and state column
city_state_df = df['NAME'].str.split(',', expand=True)

#added the place column to the new city state df
city_state_df['Places'] = df['place']

#full list of cities in the new df
city_state_df.columns = ['City/Town', 'State', 'Places']

#removed the word'city' from the City/Town Column
city_state_df['City/Town'] = city_state_df['City/Town'].str.replace(' city', '', case=False)

#removed the word'town' from the City/Town Column
city_state_df['City/Town'] = city_state_df['City/Town'].str.replace(' town', '', case=False)

#removed the word'town' from the City/Town Column
city_state_df['City/Town'] = city_state_df['City/Town'].str.replace(' CDP', '', case=False)
# city_state_df['City/Town'] = city_state_df['City/Town'].str.replace(r'\s*(city|town)\s*', '', case=False)

#Reorderd the columns
state_city_df = city_state_df[['State', 'City/Town', 'Places']]

state_city_df.head()

#exported the cites file from the Census data

# state_city_df.to_csv('../data/state_cities.csv', index=False)
# state_city_df.drop(1, axis=0)
# city_state_df['City/Town'].drop_duplicates
# city_state_df.head()
# state_group_count = city_state_df.groupby(city_state_df['State']).count()
# city_state_df.sort_values(by='State')


In [61]:
state_city_df.head()

,State,City/Town,Places
0,Alabama,Auburn,3076
1,Alabama,Birmingham,7000
2,Alabama,Dothan,21184
3,Alabama,Hoover,35896
4,Alabama,Huntsville,37000


In [ ]:
radius = # YOUR CODE HERE
params = {
    # YOUR CODE HERE
}



print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = # YOUR CODE HERE
    params["bias"] = # YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df